In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../real_monica/data.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(46, 47), dtype=bool)

In [6]:
motion.shape

(5529, 48)

In [7]:
motion[:50, 1]

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2])

In [8]:
contacts.shape

(5529, 2)

In [9]:
contacts[0:5]

array([[False, False],
       [ True, False],
       [False,  True],
       [ True, False],
       [False,  True]])

In [10]:
motion[0]

array([ 1.67561612e+09,  1.00000000e-01,  0.00000000e+00,  0.00000000e+00,
        2.20218942e-01,  1.28156364e-01, -3.79664570e-01,  2.50404894e-01,
       -1.29626960e-01, -3.77603322e-01, -2.49196276e-01,  1.29706442e-01,
       -3.83550644e-01, -2.75959224e-01, -1.30796552e-01, -3.82629216e-01,
        2.70977259e+00,  1.16201282e+00,  3.74220192e-01,  8.41218606e-02,
       -4.70607728e-03,  8.78726691e-02,  4.04834114e-02, -1.11447081e-01,
       -4.68525977e-04, -1.83918811e-02, -8.77368897e-02,  4.61280271e-02,
        1.03009783e-01,  1.05129434e-02,  4.17001754e-01,  4.04417783e-01,
       -3.86051983e-02,  6.37666732e-02, -6.84953667e-03,  1.16000000e+02,
        6.40000000e+01,  8.70000000e+01,  7.40000000e+01, -8.40414374e-04,
       -5.61846653e-04,  9.31148469e-01, -3.64638895e-01, -4.33430600e-04,
        1.97484461e-03, -2.39510274e+00,             nan,             nan])

## Height-Force plots

In [11]:
def get_specific_cmd(dataset, fwd, side, rot):
    if abs(fwd): 
        return np.where(dataset[:, 1] > 0)[0] if fwd > 0 else np.where(dataset[:, 1] < 0)[0]
    if abs(side): 
        return np.where(dataset[:, 2] > 0)[0] if side > 0 else np.where(dataset[:, 2] < 0)[0]
    if abs(rot): 
        return np.where(dataset[:, 3] > 0)[0] if rot > 0 else np.where(dataset[:, 3] < 0)[0]

In [12]:
def get_swinging_motions(dataset, height=1):
    # rf min height (i.e swining motion)
    fl_min_height = np.where(dataset[:, 6] < height)[0]
    fr_min_height = np.where(dataset[:, 9] < height)[0]
    rl_min_height = np.where(dataset[:, 12] < height)[0]
    rr_min_height = np.where(dataset[:, 15] < height)[0]
            
    return fl_min_height,fr_min_height,rl_min_height,rr_min_height

# Dataset Preparation

In [13]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [14]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [16]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [17]:
def create_acceleration_com_dataset(dataset, footsteps, motion=None, debug=False):
    idx = 1
    inputs = []
    labels = []
    
    while idx < len(footsteps):
        if idx > 0:
            # Compute time difference footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            # Round velocity array
            dataset[footsteps[idx-1], 1:4] = np.around(dataset[footsteps[idx-1], 1:4], decimals=1)
            dataset[footsteps[idx], 1:4] = np.around(dataset[footsteps[idx], 1:4], decimals=1)
            
            if time_difference < 0.4:
                fl_rr_moving = contacts[footsteps[idx], 0]
                fr_rl_moving = contacts[footsteps[idx], 1]

                if fl_rr_moving == fr_rl_moving:
                    print("Invalid footstep")
                    idx += 1
                    continue

                # Rotation matrices
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 39:43])
                #print(R_curr)

                # Retrieve base poses in world frame
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]
                
                # Compute base displacement
                world_displacement = curr_base - prev_base
                base_displacement = np.dot(R_curr.T, world_displacement)
                
                #print(dataset[footsteps[idx], 39:43])
                #print(dataset[footsteps[idx], 1], curr_base[0], prev_base[0], world_displacement[0], base_displacement[0])
                
                # Compute yaw displacement
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 39:43])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 39:43])
                yaw = curr_yaw - prev_yaw
                
                #print(dataset[footsteps[idx-1], 19], ",", dataset[footsteps[idx], 1], ",", dataset[footsteps[idx], 19])
                #print("\n")
                
                #print(world_displacement[0], base_displacement[0])
                
                inputs.append(dataset[footsteps[idx-2], 1:2].tolist() +
                              dataset[footsteps[idx-1], 1:2].tolist() +
                              dataset[footsteps[idx], 1:2].tolist() +
                              dataset[footsteps[idx-2], 19:20].tolist() +
                              dataset[footsteps[idx-1], 19:20].tolist() +
                              dataset[footsteps[idx-1], 4:16].tolist() +
                              [fl_rr_moving, fr_rl_moving]) # swiging booleans
                
                labels.append([base_displacement[0], base_displacement[1], yaw])
        idx += 1
                
    return np.array(inputs, dtype=object), np.array(labels, dtype=object)

# Stack datasets
X_com, Y_com = create_acceleration_com_dataset(motion, [x for x in range(len(motion))])
print(X_com.shape)
print(Y_com.shape)

(5527, 19)
(5527, 3)


In [18]:
%store X_com
%store Y_com

Stored 'X_com' (ndarray)
Stored 'Y_com' (ndarray)


In [20]:
X_com[0, :]

array([0.1, 0.0, 0.0, 0.0841218605638, -0.00470607727766, 0.0878726691008,
       0.0404834114015, 0.220218941569, 0.128156363964, -0.379664570093,
       0.250404894352, -0.129626959562, -0.377603322268, -0.249196276069,
       0.129706442356, -0.383550643921, -0.275959223509, -0.130796551704,
       -0.382629215717, True, False], dtype=object)

In [23]:
dic = dict()
for x in range(len(X_com)):
    key = str(round(X_com[x, 0], 1)) + str(round(X_com[x, 3], 3))
    if key in dic:
        dic[key][0].append(Y_com[x, 0])
        dic[key][1].append(X_com[x, 3])
        dic[key][2].append([X_com[x, 7], 
                            X_com[x, 10], 
                            X_com[x, 13], 
                            X_com[x, 16]])
        dic[key][3].append(X_com[x, -1])
    else:
        dic[key] = [[Y_com[x, 0]], 
                    [X_com[x, 3]],
                    [[X_com[x, 7], 
                     X_com[x, 10], 
                     X_com[x, 13], 
                     X_com[x, 16]]],
                     [X_com[x, -1]]]

In [24]:
dic.keys()

dict_keys(['0.10.125', '0.10.102', '0.10.113', '0.10.089', '0.10.119', '0.10.074', '0.10.055', '0.10.118', '0.10.094', '0.10.112', '0.10.088', '0.10.104', '0.10.078', '0.10.124', '0.10.069', '0.10.12', '0.10.087', '0.10.13', '0.10.085', '0.10.103', '0.10.091', '0.10.063', '0.10.1', '0.10.097', '0.10.076', '0.10.099', '0.10.077', '0.10.101', '0.10.116', '0.10.081', '0.10.09', '0.10.109', '0.10.068', '0.10.073', '0.10.11', '0.10.106', '0.10.107', '0.10.096', '0.10.083', '0.10.133', '0.10.086', '0.10.115', '0.10.111', '0.10.06', '0.20.12', '0.20.103', '0.20.226', '0.20.223', '0.20.217', '0.20.179', '0.20.203', '0.20.208', '0.20.229', '0.20.168', '0.20.211', '0.20.189', '0.20.224', '0.20.213', '0.20.186', '0.20.231', '0.20.185', '0.20.21', '0.20.228', '0.20.197', '0.20.206', '0.20.222', '0.20.175', '0.20.225', '0.20.202', '0.20.207', '0.20.219', '0.20.181', '0.20.218', '0.20.174', '0.00.192', '0.00.102', '0.0-0.003', '0.0-0.051', '0.00.002', '0.0-0.053', '0.00.001', '0.0-0.014', '0.00.018'

In [25]:
for key in dic.keys():
    #print(f"{key[:3]} -> {key[3:]}. Mean: {np.round(np.mean(dic[key][0]), 4)}. Std: {np.round(np.std(dic[key][0]), 3)}.")
    if float(key[:3]) == 0.3 and np.isclose(float(key[3:]), 0.44, 0.05):
        for x in range(len(dic[key][0])):
            print(f"Actual velocity: {dic[key][1][x]}. Prediction: {dic[key][0][x]}. Feet positions: {dic[key][2][x]}")

In [26]:
for key in dic.keys():
    for x in range(len(dic[key][0])):
        print(f"{key[:3]}->{key[3:]}. Disp: {np.round(dic[key][0][x], 6)}. CoM Vel: {np.round(dic[key][1][x], 3)}.")

0.1->0.125. Disp: 0.03306. CoM Vel: 0.125.
0.1->0.102. Disp: 0.026439. CoM Vel: 0.102.
0.1->0.102. Disp: 0.02493. CoM Vel: 0.102.
0.1->0.113. Disp: 0.029767. CoM Vel: 0.113.
0.1->0.113. Disp: 0.028517. CoM Vel: 0.113.
0.1->0.089. Disp: 0.025462. CoM Vel: 0.089.
0.1->0.089. Disp: 0.023749. CoM Vel: 0.089.
0.1->0.119. Disp: 0.028897. CoM Vel: 0.119.
0.1->0.119. Disp: 0.024091. CoM Vel: 0.119.
0.1->0.074. Disp: 0.025263. CoM Vel: 0.074.
0.1->0.055. Disp: 0.028165. CoM Vel: 0.055.
0.1->0.118. Disp: 0.029416. CoM Vel: 0.118.
0.1->0.118. Disp: 0.026667. CoM Vel: 0.118.
0.1->0.094. Disp: 0.028068. CoM Vel: 0.094.
0.1->0.094. Disp: 0.027727. CoM Vel: 0.094.
0.1->0.112. Disp: 0.027245. CoM Vel: 0.112.
0.1->0.088. Disp: 0.027132. CoM Vel: 0.088.
0.1->0.088. Disp: 0.029299. CoM Vel: 0.088.
0.1->0.088. Disp: 0.026518. CoM Vel: 0.088.
0.1->0.104. Disp: 0.026069. CoM Vel: 0.104.
0.1->0.078. Disp: 0.027078. CoM Vel: 0.078.
0.1->0.078. Disp: 0.027959. CoM Vel: 0.078.
0.1->0.124. Disp: 0.025159. CoM Ve

In [27]:
for key in dic.keys():
    print(f"{key[:3]}->{key[3:]}. Mean: {np.round(np.mean(dic[key][0]), 3)}. Std: {np.round(np.std(dic[key][0]), 3)}.")

0.1->0.125. Mean: 0.033. Std: 0.0.
0.1->0.102. Mean: 0.026. Std: 0.001.
0.1->0.113. Mean: 0.029. Std: 0.001.
0.1->0.089. Mean: 0.025. Std: 0.001.
0.1->0.119. Mean: 0.026. Std: 0.002.
0.1->0.074. Mean: 0.025. Std: 0.0.
0.1->0.055. Mean: 0.028. Std: 0.0.
0.1->0.118. Mean: 0.028. Std: 0.001.
0.1->0.094. Mean: 0.028. Std: 0.0.
0.1->0.112. Mean: 0.027. Std: 0.0.
0.1->0.088. Mean: 0.028. Std: 0.001.
0.1->0.104. Mean: 0.026. Std: 0.0.
0.1->0.078. Mean: 0.028. Std: 0.0.
0.1->0.124. Mean: 0.025. Std: 0.0.
0.1->0.069. Mean: 0.024. Std: 0.0.
0.1->0.12. Mean: 0.028. Std: 0.001.
0.1->0.087. Mean: 0.029. Std: 0.005.
0.1->0.13. Mean: 0.03. Std: 0.0.
0.1->0.085. Mean: 0.029. Std: 0.002.
0.1->0.103. Mean: 0.029. Std: 0.003.
0.1->0.091. Mean: 0.026. Std: 0.0.
0.1->0.063. Mean: 0.026. Std: 0.0.
0.1->0.1. Mean: 0.029. Std: 0.002.
0.1->0.097. Mean: 0.029. Std: 0.0.
0.1->0.076. Mean: 0.026. Std: 0.0.
0.1->0.099. Mean: 0.03. Std: 0.0.
0.1->0.077. Mean: 0.028. Std: 0.001.
0.1->0.101. Mean: 0.026. Std: 0.0.
0.

In [28]:
dic.keys()

dict_keys(['0.10.125', '0.10.102', '0.10.113', '0.10.089', '0.10.119', '0.10.074', '0.10.055', '0.10.118', '0.10.094', '0.10.112', '0.10.088', '0.10.104', '0.10.078', '0.10.124', '0.10.069', '0.10.12', '0.10.087', '0.10.13', '0.10.085', '0.10.103', '0.10.091', '0.10.063', '0.10.1', '0.10.097', '0.10.076', '0.10.099', '0.10.077', '0.10.101', '0.10.116', '0.10.081', '0.10.09', '0.10.109', '0.10.068', '0.10.073', '0.10.11', '0.10.106', '0.10.107', '0.10.096', '0.10.083', '0.10.133', '0.10.086', '0.10.115', '0.10.111', '0.10.06', '0.20.12', '0.20.103', '0.20.226', '0.20.223', '0.20.217', '0.20.179', '0.20.203', '0.20.208', '0.20.229', '0.20.168', '0.20.211', '0.20.189', '0.20.224', '0.20.213', '0.20.186', '0.20.231', '0.20.185', '0.20.21', '0.20.228', '0.20.197', '0.20.206', '0.20.222', '0.20.175', '0.20.225', '0.20.202', '0.20.207', '0.20.219', '0.20.181', '0.20.218', '0.20.174', '0.00.192', '0.00.102', '0.0-0.003', '0.0-0.051', '0.00.002', '0.0-0.053', '0.00.001', '0.0-0.014', '0.00.018'

In [29]:
x, y, e = [], [], []

for vel in np.round(np.arange(0.0, 1.1, 0.1), 2):
    start = vel
    key = str(start)+str(vel)
    print(f"{start} -> {vel}. Mean: {np.round(np.mean(dic[key][0]), 3)}. Std: {np.round(np.std(dic[key][0]), 3)}")
    x.append(vel)
    y.append(float(np.mean(dic[key][0])))
    e.append(float(np.std(dic[key][0])))

plt.errorbar(x, y, e, linestyle='None', marker='^')

0.0 -> 0.0. Mean: -0.001. Std: 0.003
0.1 -> 0.1. Mean: 0.029. Std: 0.002
0.2 -> 0.2. Mean: 0.054. Std: 0.0


KeyError: '0.30.3'